In [1]:
import torch 
import torch.nn as nn
from torch.nn import functional as F

import sys
sys.path.append('..')
from d2l_helpers import *

In [2]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X 
        return F.relu(Y)

In [3]:
blk = Residual(3, 3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

In [4]:
blk = Residual(3, 6, use_1x1conv=True, strides=2)
Y = blk(X)
Y.shape

torch.Size([4, 6, 3, 3])

In [5]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1, 1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [6]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


In [7]:
device = get_device()
lr, num_epochs, batch_size = 0.05, 10, 256
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=96)
train_cnn(net, train_iter, test_iter, num_epochs, lr, device)

training on mps
epoch 1, iter 47/235
epoch 1, iter 94/235
epoch 1, iter 141/235
epoch 1, iter 188/235
epoch 1, iter 235/235
loss 0.483, train acc 0.828, test acc 0.876
epoch 2, iter 47/235
epoch 2, iter 94/235
epoch 2, iter 141/235
epoch 2, iter 188/235
epoch 2, iter 235/235
loss 0.256, train acc 0.906, test acc 0.889
epoch 3, iter 47/235
epoch 3, iter 94/235
epoch 3, iter 141/235
epoch 3, iter 188/235
epoch 3, iter 235/235
loss 0.194, train acc 0.929, test acc 0.840
epoch 4, iter 47/235
epoch 4, iter 94/235
epoch 4, iter 141/235
epoch 4, iter 188/235
epoch 4, iter 235/235
loss 0.146, train acc 0.947, test acc 0.855
epoch 5, iter 47/235
epoch 5, iter 94/235
epoch 5, iter 141/235
epoch 5, iter 188/235
epoch 5, iter 235/235
loss 0.112, train acc 0.960, test acc 0.903
epoch 6, iter 47/235
epoch 6, iter 94/235
epoch 6, iter 141/235
epoch 6, iter 188/235
epoch 6, iter 235/235
loss 0.080, train acc 0.973, test acc 0.852
epoch 7, iter 47/235
epoch 7, iter 94/235
epoch 7, iter 141/235
epoch 7,